> **DO NOT EDIT IF INSIDE annadl_f19 folder**


# Week 5: Recurrent neural networks

Text, speech, weather, sensor output and video are but a few examples of the many types of data that is inherently sequential. So how does one predict the next word in a sentence, future temperatures or missing video frames? Using **recurrent neural networks** (RNNs)!

In [3]:
%matplotlib inline
%load_ext tensorboard
import numpy as np
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import requests
import datetime
import keras

Using TensorFlow backend.


## Exercises

#### Modeling text

Text prediction is a good place to start when learning about RNNs, because most of us humans have a pretty well
optimized inner model for text prediction ourselves. We can, therefore, easily assess the performance of a neural
network in executing this task.

Below is some code that loads the screenplay for Tarantino's 1994 film 'Pulp Fiction'. I recommend reading through the
first 20 lines or so to get a feeling for the language and style used (and enjoy probably the best written screenplay
in the history of film).

In [11]:
response = requests.get("http://www.dailyscript.com/scripts/pulp_fiction.html")
text = BeautifulSoup(response.content, "html.parser").getText()
print(text[:2000])



"PULP FICTION" -- by Quentin Tarantino & Roger Avary


                                      "PULP FICTION"

                                            By

                             Quentin Tarantino & Roger Avary

                

               PULP [pulp] n.

               1. A soft, moist, shapeless mass or matter.

               2. A magazine or book containing lurid subject matter and 
               being characteristically printed on rough, unfinished paper.

               American Heritage Dictionary: New College Edition

               INT. COFFEE SHOP – MORNING

               A normal Denny's, Spires-like coffee shop in Los Angeles. 
               It's about 9:00 in the morning. While the place isn't jammed, 
               there's a healthy number of people drinking coffee, munching 
               on bacon and eating eggs.

               Two of these people are a YOUNG MAN and a YOUNG WOMAN. The 
               Young Man has a slight working-class English acce

> **Ex. 5.1.1:** What is the most used symbol in this screenplay and what accuracy would a model constantly predicting this symbol obtain? In other words, what is the "baseline accuracy"?

I've adapted some code for text generation from [this Keras example](https://keras.io/examples/lstm_text_generation/).
I've inserted some questions in the code (look for `Q:`) for you to answer in the exercise below.

In [20]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
# makes a list of all the characters then makes to dict pairs where one dict is a character to index and the
# other is index to character. This is to help the algorithm see the word order and commonality of certain 
# characters
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
# seqlen is the length of the sequence of text grabed at each iteration, the step parameter skips over to the 
# next seqlen of characters. Each seqlen of text is evaluated as a sentence and appended to sentences
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
# x is a character and y is the character after x. They are of the dimensions len(sentences) and seqlen
# because we input all the sentences into the matrix and the sentences are seqlen
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    # sets x as a character and y as the character after it
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
# the return sequences tells whether to return the last output in the output sequence, or the full sequence.
# the final dense layer maps to each single unique character found in the input sequence
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)



log_dir="./logs/wk5_1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)


In [21]:
model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback, tensorboard_callback])

Epoch 1/50
7614/7614 [==============================] - 8s 1ms/step - loss: 2.2236 - categorical_crossentropy: 2.2236 - accuracy: 0.5426

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ee.

               The Young Man goes"
ee.

               The Young Man goes                                                                                                                                                                                                                                                                                                                                                                                                                
----- diversity: 0.5
----- Generating with seed: "ee.

               The Young Man goes"
ee.

               The Young Man goes roe  o                                                                                                                                                                 

                                                                                                                                                                                              BUTCH
                                                                                                                                                                       
----- diversity: 1.0
----- Generating with seed: "Brett)
                         Now I'm"
Brett)
                         Now I'me wa coneme.

                                  The mannempliced of fe't and 
                   Vanceniod kace.

                                      creap-atha callan, suref)"Breeve 
                                JULES
                                  9usules out!

                          Butch kingy and I dnough stantersheemore.

                                    Dood, to tex
Epoch 6/50
7614/7614 [==============================] - 6s 789us/step - loss: 0.9963 - categorical_crossentropy: 0.9

                         needle. Five out a parting a table a pot the 
                                                                                                                                                                                                                                                                                                                                                                             
----- diversity: 0.5
----- Generating with seed: " 
                         needle. Five"
 
                         needle. Fiven and way ready and say 
                         The warked Marsellus out a fight out of the boot 
                                                                                                                                                                                                                                                                                                          
----- diversity: 1.0
----- Generat

                                          VINCENT
                         Larks if a copbyess.

                                             BRETT
                         Vince, kitchag. He told his way and 
                         mor eadina to calley a barbaty. CCmp. And the wall of Fabbel Nord onomie, Maynard instadds, Marsellusn to 
               Marsellus 
Epoch 15/50
7614/7614 [==============================] - 6s 773us/step - loss: 0.7829 - categorical_crossentropy: 0.7829 - accuracy: 0.7807

----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "

                                    "


                                                                                                                                                                                                                                                                                                                                                                         

                                              JULES
                         What did you walk the past be a phin 
                         deed a culler a bitch stacts the shake 
                         but you guy the open street. I'm 
                         sincent stopped the shot.

                                                    JULES
                         What don't you gotta know. They comes 
        
----- diversity: 1.0
----- Generating with seed: "ncakes?

                             "
ncakes?

                                     VINCENT
                         She's securaina bat she hagare.

               Vince I sure head, don't arright no 
                         go out off got my all the car as. We 
                   the blooding, in martrot-the-forsterd kneckes through a huss in the 
               Spakes. Reated bach this shit 
                         place, STINNG his stile scarition.

       
Epoch 20/50
7614/7614 [==============================] - 7

...

               her ears given in the arril...

               Jules stles and takes to the dumbiercel..............

               Jules CHINGS, by side, ever here. Butheresser.

               Three black cartia latch Marisy. And her 
                         for her instently that for ther body then.

                                          JULES
                         Shit inside wife, Marbilly sugarns 
Epoch 29/50
7614/7614 [==============================] - 6s 791us/step - loss: 0.6824 - categorical_crossentropy: 0.6824 - accuracy: 0.8076 2s - loss: 0.6690 - categorical_crossentr

----- Generating text after Epoch: 28
----- diversity: 0.2
----- Generating with seed: "n a table catching a few 
             "
n a table catching a few 
                         you want to be and you thought it is 
                         you want to it.

                                                                                                                                              


                                       JULES
                         I'm not a cocked.

                                                                                                                                                                                                                                                                                                                  
----- diversity: 1.0
----- Generating with seed: "
                         Que hora es?
"

                         Que hora es?

                                       BUTCH (O.S.)
                         INT. CHOT –" LOUPS oFF.!

               He like it's not nours. It's unherood, a rackiur to you're it a Dan evect.

                                         BUTCH
                         Horror. Overy'Derring it sellua-sayul of at 
                         Kook.

                                     FABIENNE
Epoch 34/50
7614/7614 [==============================] - 6s 789us/step - loss: 0.66

h SCREAMS... with one mighty SWING, SLASEL 
                         the should be never say your bullets. The 
                         the same not to go to ever here to 
                         the same to say. And he was in the 
                         the should be come to said a smake a books. I 
                         the same and stand in front of the fuckin' 
                         the should be come to said in the 
----- diversity: 0.5
----- Generating with seed: "h SCREAMS... with one mighty SWING, SLAS"
h SCREAMS... with one mighty SWING, SLASS Rody 
                         hand caught on the same. It's 
                                                                                                                                   BUTCH
                         What the fuck headd is and Jules? Hell a 
                         so same touch the backser see BREAT Goren 
                         in the fuckin' bit to man are beat
----- diversity: 1.0
----- Generating

                         you still wead upco the broth that fucked himself do 
                         The Cluen his quitcosteretirg)
                         One on the hash. It's reacking, Jules. Clming its dead into the moment 
                         that, bringo this pawnshop fame of charish reach 
                         my watch the place.

                                
Epoch 43/50
7614/7614 [==============================] - 7s 915us/step - loss: 0.6431 - categorical_crossentropy: 0.6431 - accuracy: 0.8172

----- Generating text after Epoch: 42
----- diversity: 0.2
----- Generating with seed: "police cars with their SIRENS BLARING ra"
police cars with their SIRENS BLARING rasuble. The 
                         the only that and you wanna know what to 
                         the bag.

                                     BUTCH
                         What does to a big furger and start and sead and starts to 
                         the way the way the world on the fir

        us is Butch's French girlfriend, the same from the fuck out of the doorbeet and she coulda 
                         they should be fuckin' die. You steath 
                         they pot, I want to the one was because 
                         each other, man, I don't wanna be the 
                         they get a pretting what he want to the descriped to the light.

                                     JULES
     
----- diversity: 1.0
----- Generating with seed: "        us is Butch's French girlfriend,"
        us is Butch's French girlfriend, and Vincent quick a dut.

               Bacro becie "Malosn lot of dibloows with Cocidy up.

                                     YOUNG WOMAN
                         Eomeroone.

                                     BUTCH
                              (over intere teens, I wouldn't 
                         startlingu in front of Marvin)

               LIVY BOOMBB-TMENNG) – MORNING

Epoch 48/50
7614/7614 [======================

> **Ex. 5.1.2**: Add a callback for Tensorboard, so you can log the training process. Start training the network (takes ~10 minutes on my computer). While it's running move on to the next question.

In [26]:
%tensorboard --logdir logs/wk5_1

> **Ex. 5.1.3**: Answer the questions in the code above (look for code comments starting with `Q:`).

> **Ex. 5.1.4**: Did the network finish training? Consider the generated text across epochs.
1. In the early batches (0-10), the generated text looks very bad. Can you explain why the low diversity generated text contains almost only the symbol " " (that is, spaces)?
2. The high diversity generated text is messed up too, but in a different way. Explain how.
3. In later batches (20-30) what do you notice is off about the low diversity generated text?

1. Early batches have low diversity and a lot of " " characters because the empty character is the most common one to appear in the text sequence, therefore the neural network easily predicts it as the most likely output after most characters.
2. The higher diversity text has more variation of characters, but they don't necessarily form coherent sentences
3. The low diversity text in the higher epochs still have a lot of empty space characters, but they also tend to have very common sentence structure (like repeating same words or similar spelled words).

> **Ex. 5.1.5**: For the network trained over all 50 epochs, generate a longer piece of text
(say 5000 symbols long). Use the sentence `text[1486:1526]` as seed (starts with 'YOUNG MAN' ends with 'No, ')
and set diversity to 0.5.
Describe what features of the screenplay and language in general that the network learned in only 50 epochs.
Also describe what serious mistakes it makes.

In [25]:
start_index = 1486
seqlen = 40
diversity = 0.5

generated = ''
sentence = text[start_index: start_index + seqlen]
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(400):
    x_pred = np.zeros((1, seqlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
    preds = model.predict(x_pred, verbose=0)
    next_index = sample(preds[0, -1], diversity)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()


----- Generating with seed: "YOUNG MAN
                         No, "
YOUNG MAN
                         No, it's a fact on the toor to it what I 
                                     JULES
                         Well, if it with this tomaticn.

                                       VINCENT
                         And withouster mind money on his as in 
                         to be they're a man, while I was to 
                         still can toward Vincent's prises the bathroom soulda 


> **Ex. 5.1.6**: Do the same as above, but for 40 random letters (e.g. smash away on your keyboard) as seed. What happens? Can you explain why?

In [23]:
start_index = 0
seqlen = 40
diversity = 0.5

random_text = 'asdnkjsanglkseaklgj asjdbmnsavd hjgs eyfgashbdvnsdabkyasgb nbsdh bajsbcmnsdbgaksbfuidfhvuiasehf rsdgzsdvbj hfvhB Khv kzsdb,ds'

generated = ''
sentence = random_text[start_index: start_index + seqlen]
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(400):
    x_pred = np.zeros((1, seqlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
    preds = model.predict(x_pred, verbose=0)
    next_index = sample(preds[0, -1], diversity)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

----- Generating with seed: "asdnkjsanglkseaklgj asjdbmnsavd hjgs eyf"
asdnkjsanglkseaklgj asjdbmnsavd hjgs eyfing the 
                         and toward the wall.

                                       VINCENT
                                     FABIENNE
                         I don't mean I have a fuckin' gotto 
                         fuck does a smoke.

                                                                                    JULES
                         Yeah, in the front o


> **Challenge** Download [this](https://www.yelp.com/dataset/download) Yelp dataset and train a model that predicts rating given a review text!